In [ ]:
# Prerequisites
# Configure custom environment and add two libs
# - openai
# - googlemaps

# Create a lakehouse with the name "LAKEHOUSE"
# Attach that lakehouse as a default to each notebook

In [26]:
%run ReviseCare_Secrets

StatementMeta(, ee7945a0-874e-45e2-bb36-a12397e6bb55, 30, Finished, Available, Finished)

Variables GMAP_API_KEY, OPENAI_API_KEY, AZURE_OPEN_AI_KEY, AZURE_OPEN_AI_ENDPOINT were initialized


In [14]:
import googlemaps
import time
import json

from openai import OpenAI
from openai import AzureOpenAI

StatementMeta(, ee7945a0-874e-45e2-bb36-a12397e6bb55, 18, Finished, Available, Finished)

In [ ]:
# Initialize the client with your API key
gmaps = googlemaps.Client(key=GMAP_API_KEY)
print("Google client gmaps was initialized")

In [ ]:
def list_to_dataframe(data: list):
    json_data = []

    for i in data:
        jsonData = json.dumps(i)
        json_data.append(jsonData)

    jsonRDD = sc.parallelize(json_data)

    df = spark.read.json(jsonRDD)

    return df

print("Function list_to_dataframe() initialized")

In [ ]:
def get_ids_from_config(hospitals_config: str) -> str:
    place_ids = []
    
    for r in hospitals_config.split('\n'):
        if r.strip() == '':
            continue
        place_name, place_id = r.split(':')
        place_ids.append(place_id)

    return place_ids

print("Function get_ids_from_config() initialized")

In [ ]:
def get_hospital_details(place_id):
    # Get detailed information about a hospital using the Place Details API
    fields = ["name", "business_status", "formatted_address", "place_id", "review", "rating", "user_ratings_total"]
    
    details = gmaps.place(place_id=place_id, fields=fields)
    return details['result']

print("Function get_hospital_details() initialized")

In [30]:
def openai_get_answer(instructions: str, question: str, data: str):
    
    if len(AZURE_OPEN_AI_KEY)>0:
        API_VERSION = "2024-02-01"
        MODEL_NAME = "gpt-4o"

        client = AzureOpenAI(
            azure_endpoint=AZURE_OPEN_AI_ENDPOINT,
            api_key=AZURE_OPEN_AI_KEY,
            api_version=API_VERSION,
        )
    else:
        client = OpenAI(api_key = OPENAI_API_KEY)
        MODEL_NAME = "gpt-4o-mini"
    
    '''
    instructions: should limit the answers
    question: formulates the main question
    data: provides the context
    '''
    
    MESSAGES = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": question},
        {"role": "user", "content": f"data for analysis: {data}"},
    ]

    completion = client.chat.completions.create(
        model=MODEL_NAME,
        messages=MESSAGES
    )

    answer = completion.choices[0].message.content

    return answer

from pyspark.sql.functions import udf, lit
from pyspark.sql.types import StringType

openai_udf = udf(openai_get_answer, StringType())
spark.udf.register("openai_udf", openai_get_answer, StringType())

print("Function openai_get_answer() initialized")
# print(openai_get_answer(instructions='Help', question='What is the smallest country?', data='no data'))

StatementMeta(, ee7945a0-874e-45e2-bb36-a12397e6bb55, 34, Finished, Available, Finished)

Function openai_get_answer() initialized
